### 1. Импорт необходимых библиотек

In [17]:
import os
import re
import jiwer
import pandas as pd
import Levenshtein
from num2words import num2words
from pymorphy2 import MorphAnalyzer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

### 2. Считывание данных

Прочитаем данные из файла с транскрибацией Whisper в датафрейм. Для начала оставим данные в "сыром" виде

In [21]:
with open('Транскрипт Whisper.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

In [23]:
df = pd.DataFrame({'raw': [line.strip() for line in lines if line.strip()]})
df.head(3)

,raw
0,"###1 [0:00:00 --> 0:00:03] - Здравствуйте, в..."
1,###1 [0:00:04 --> 0:00:10] - Уважаемый клиен...
2,###1 [0:00:11 --> 0:00:16] - Вы можете сформ...


Рассмотрим отдельную строку датафрейма:

In [24]:
df.iloc[1]['raw']

'###1 [0:00:04 --> 0:00:10]  -  Уважаемый клиент, информируем вас, что срок предоставления актов сверки и закрывающих документов составляет 7 рабочих дней.'

Выполним разделение считанных строк данных на столбцы, используя регулярное выражение

In [ ]:
pattern = r'###(\d+)\s+\[(.*?)\s*-->\s*(.*?)\]\s*-\s*(.*)'

In [26]:
whisper = df['raw'].str.extract(pattern)
whisper.head(3)

,0,1,2,3
0,1,0:00:00,0:00:03,"Здравствуйте, вы позвонили в центр сопровожден..."
1,1,0:00:04,0:00:10,"Уважаемый клиент, информируем вас, что срок пр..."
2,1,0:00:11,0:00:16,"Вы можете сформировать документы прямо сейчас,..."


Переименуем столбцы в соответствии с содержимым

In [27]:
whisper.columns=['speaker', 'start_time', 'end_time', 'text']
whisper.head(3)

,speaker,start_time,end_time,text
0,1,0:00:00,0:00:03,"Здравствуйте, вы позвонили в центр сопровожден..."
1,1,0:00:04,0:00:10,"Уважаемый клиент, информируем вас, что срок пр..."
2,1,0:00:11,0:00:16,"Вы можете сформировать документы прямо сейчас,..."


Рассмотрим распределение данных

In [28]:
# Проверка на пропущенные значения
whisper.isnull().sum()

speaker       0
start_time    0
end_time      0
text          0
dtype: int64

In [29]:
# Проверка на дубликаты
whisper.duplicated().sum()

0

In [30]:
# Количество строк
whisper.shape

(70, 4)

#### Функция для всей обработки

Запишем функцию для выполнения необходимой предобработки и загрузки данных

In [55]:
def process_file(file_path: str) -> pd.DataFrame:
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    df = pd.DataFrame({'raw': [line.strip() for line in lines if line.strip()]})
    pattern = r'###(\d+)\s+\[(.*?)\s*-->\s*(.*?)\]\s*-\s*(.*)'

    df = (
        df['raw']
        .str.extract(pattern)
        .rename(columns={
            0: 'speaker',
            1: 'start_time',
            2: 'end_time',
            3: 'text'
        })
    )

    return df

Загрузим данные эталонной и Yandex транскрибации

In [56]:
yandex = process_file('Транскрипт Yandex.txt')
yandex.head(3)

,speaker,start_time,end_time,text
0,1,0:00:00.199000,0:00:41.590000,"Здравствуйте, вы позвонили в Центр сопровожден..."
1,1,0:00:44.079000,0:01:05.519000,"Нужно обновить программу, нажмите, 1 не получа..."
2,1,0:01:15.640000,0:02:07.150000,Мы не получили действительный ответ. Подождите...


In [58]:
reference = process_file('Эталонный транскрипт.txt')
reference.head(3)

,speaker,start_time,end_time,text
0,1,0:00:00,0:00:03,Здравствуйте. Вы позвонили в центр сопровожден...
1,1,0:00:03,0:00:10,"Уважаемый клиент, информируем вас, что срок пр..."
2,1,0:00:11,0:00:16,"Вы можете сформировать документы прямо сейчас,..."


In [59]:
print('Строк в Whisper:', whisper.shape[0])
print('Строк в Yandex:', yandex.shape[0])
print('Строк в Reference:', reference.shape[0])

Строк в Whisper: 70
Строк в Yandex: 30
Строк в Reference: 59


#### Альтернативный способ считывания данных

Если изначально ознакомиться с содержимым файлов (до загрузки данных в pandas), можно заметить, что текстовая часть транскрибации всегда начинается после одного и того же сочетания символов. Так же видно, что результаты транскрибации для Whisper и Yandex существенно различаются по разбиению текста на временные интервалы и количеству строк соответственно.  Учитывая это, можно произвести загрузку данных иным способом: извлечь сразу только текст, игнорируя временные метки и информацию о текущем участнике разговора

In [60]:
delimiter = ']  -  '   # Разделитель, после которого начинается текст
texts = []

with open('Транскрипт Whisper.txt', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue   # Пропускаем пустые строки и переходим к следующей итерации цикла
        # Разбиваем строку по разделителю, извлекаем текст после него
        parts = line.split(delimiter, 1)
        text = parts[1].strip()
        texts.append({'Text': text})

# Создаем DataFrame
df = pd.DataFrame(texts)
df.head(3)

,Text
0,"Здравствуйте, вы позвонили в центр сопровожден..."
1,"Уважаемый клиент, информируем вас, что срок пр..."
2,"Вы можете сформировать документы прямо сейчас,..."


### 3. Обработка текста

Нормализация текста для подсчёта метрик качества транскрибации может включать:
- Удаление пунктуации
- Приведение символов к нижнему регистру
- Удаление лишних пробелов
- Замена "ё" на "е"
- Замена цифровой записи числительных на буквенную

#### Своя функция

Для выполнения нормализации можно записать собственную функцию следующего вида:

In [61]:
def clean_text(text: str) -> str:
    text = re.sub(r'[^\w\s]', ' ', text).lower()   # Удаление пунктуации и приведение к нижнему регистру
    text = re.sub(r'\s+', ' ', text).strip()   # Удаление лишних пробелов
    text = text.replace('ё', 'е')   # Замена 'ё' на 'е'
    return text

In [62]:
# Пример для визуализации работоспособности функции
row = ' Съешь ещё этих   мягких французских  булок, да выпей [же] чаю.'

print(row)
print(clean_text(row))

 Съешь ещё этих   мягких французских  булок, да выпей [же] чаю.
съешь еще этих мягких французских булок да выпей же чаю


#### Jiwer

А можно воспользоваться встроенными методами библиотеки `jiwer`, которая в дальнейшем потребуется для вычисления метрик качества транскрибации:

In [63]:
def preprocess_text(text: str) -> str:
    transformation = jiwer.Compose([
        jiwer.ToLowerCase(),
        jiwer.SubstituteRegexes({r'ё': r'е', r':': r' '}),   # Для корректной обработки сочетаний вида "1С:Управление"
        jiwer.RemovePunctuation(),
        jiwer.RemoveWhiteSpace(replace_by_space=True),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip()
    ])
    
    return transformation(text)

In [64]:
print(row)
print(preprocess_text(row))

 Съешь ещё этих   мягких французских  булок, да выпей [же] чаю.
съешь еще этих мягких французских булок да выпей же чаю


Передадим в одну строку весь текст транскрибации Whisper в предобработанном виде

In [ ]:
whisper_text = preprocess_text(' '.join(whisper['text']))

Аналогично поступим для эталонной и Yandex транскрибации

In [66]:
yandex_text = preprocess_text(' '.join(yandex['text']))
reference_text = preprocess_text(' '.join(reference['text']))

### 4. Вычисление метрик

Существует ряд общепринятых метрик для оценки точности автоматического распознавания речи.
В основном будем ориентироваться на метрики *Word Error Rate (WER)* и *Character Error Rate (CER)*, как наиболее популярные, понятные интуитивно и простые для вычисления.

#### WER

In [67]:
wer_whisper = jiwer.wer(reference_text, whisper_text)
wer_yandex = jiwer.wer(reference_text, yandex_text)

print(f'WER Whisper: {wer_whisper:.2%}')
print(f'WER Yandex: {wer_yandex:.2%}')

WER Whisper: 21.15%
WER Yandex: 29.70%


#### CER

In [68]:
cer_whisper = jiwer.cer(reference_text, whisper_text)
cer_yandex = jiwer.cer(reference_text, yandex_text)

print(f'CER Whisper: {cer_whisper:.2%}')
print(f'CER Yandex: {cer_yandex:.2%}')

CER Whisper: 13.99%
CER Yandex: 16.88%


Дополнительно можно попробовать улучшить метрики путём замены записи числительных цифрами на буквенную запись (при этом стоит учесть характерное для текста сочетание "1С", где цифра "1" является частью устойчивого обозначения – его лучше оставить без изменений)

In [ ]:
# Учтём все возможные варианты записи "1С"
reference_text_without_num = re.sub(r'\b(?<!1[СсCcSs])\d+\b', lambda x: num2words(int(x.group(0)), lang='ru'), reference_text)
whisper_text_without_num = re.sub(r'\b(?<!1[СсCcSs])\d+\b', lambda x: num2words(int(x.group(0)), lang='ru'), whisper_text)
yandex_text_without_num = re.sub(r'\b(?<!1[СсCcSs])\d+\b', lambda x: num2words(int(x.group(0)), lang='ru'), yandex_text)

In [70]:
wer_whisper_ = jiwer.wer(reference_text_without_num, whisper_text_without_num)
wer_yandex_ = jiwer.wer(reference_text_without_num, yandex_text_without_num)

print(f'WER Whisper: {wer_whisper_:.2%}')
print(f'WER Yandex: {wer_yandex_:.2%}')

WER Whisper: 21.15%
WER Yandex: 29.70%


In [71]:
cer_whisper_ = jiwer.cer(reference_text_without_num, whisper_text_without_num)
cer_yandex_ = jiwer.cer(reference_text_without_num, yandex_text_without_num)

print(f"CER Whisper: {cer_whisper_:.2%}")
print(f"CER Yandex: {cer_yandex_:.2%}")

CER Whisper: 13.74%
CER Yandex: 17.58%


Метрику CER действительно удалось немного улучшить. Метрика WER осталось той же – это объясняется тем, что обе модели (Whisper и Yandex) изначально внесли в текст распознанные числительные аналогично эталонной транскрибации

#### IWER

Помимо стандартных метрик WER и CER, существуют более адаптированные подходы. Например, для русского языка с его сложной морфологией эффективным инструментом является IWER (*Inflectional Word Error Rate, Флективная ошибка распознавания слов*). Эта метрика учитывает грамматические вариации слов: падежи, спряжения, склонения

Для расчёта метрики IWER необходимо привести слова текста к базовой форме (лемме)

In [72]:
morph = MorphAnalyzer()

In [73]:
def lemmatize_text(text: str) -> str:
    return ' '.join([morph.parse(word)[0].normal_form for word in text.split()])

In [74]:
def calculate_iwer(reference: str, hypothesis: str) -> float:
    ref_processed = lemmatize_text(reference)
    hyp_processed = lemmatize_text(hypothesis)

    return jiwer.wer(ref_processed, hyp_processed)

In [75]:
print(f'IWER Whisper: {calculate_iwer(reference_text, whisper_text):.2%}')
print(f'IWER Yandex: {calculate_iwer(reference_text, yandex_text):.2%}')

IWER Whisper: 20.73%
IWER Yandex: 29.06%


#### Другие метрики

Дополнительно рассмотрим следующие метрики:
- *MER (Match Error Rate)* – процент несоответствия слов, чувствительна к перестановкам
- *WIL (Word Information Lost)* – мера потерянной информации в словах, близка к 1 при сильной ошибке
- *WIP (Word Information Preserved)* – сколько информации сохранено (1 – идеально)
- *BLEU (Bilingual Evaluation Understudy)* – схожесть n-граммам (обычно для перевода, но применим и к транскрибации)
- *Cosine similarity* – косинусная близость текстов по TF-IDF, от 0 до 1 (1 – идентичны)
- *Levenshtein distance* – количество изменений, нужных для превращения одного текста в другой

In [83]:
# Вспомогательные функции для вычисления доп метрик
def bleu_score(reference: str, hypothesis: str) -> float:
    ref_tokens = reference.split()
    hyp_tokens = hypothesis.split()
    smoothie = SmoothingFunction().method1
    return sentence_bleu([ref_tokens], hyp_tokens, smoothing_function=smoothie)

def cosine_sim(text1: str, text2: str) -> float:
    vec = TfidfVectorizer().fit_transform([text1, text2])
    return cosine_similarity(vec[0:1], vec[1:2])[0][0]

In [80]:
whisper_measures = jiwer.compute_measures(reference_text, whisper_text)

print('Whisper additional metrics:')
print(f'MER: {whisper_measures["mer"]:.2%}')
print(f'WIL: {whisper_measures["wil"]:.2%}')
print(f'WIP: {whisper_measures["wip"]:.2f}')
print(f'BLEU: {bleu_score(reference_text, whisper_text):.2f}')
print(f'Cosine similarity: {cosine_sim(reference_text, whisper_text):.2f}')
print(f'Levenshtein distance: {Levenshtein.distance(reference_text, whisper_text)}')

Whisper additional metrics:
MER: 19.30%
WIL: 24.33%
WIP: 0.76
BLEU: 0.77
Cosine similarity: 0.93
Levenshtein distance: 445


In [81]:
yandex_measures = jiwer.compute_measures(reference_text, yandex_text)

print('Yandex additional metrics:')
print(f'MER: {yandex_measures["mer"]:.2%}')
print(f'WIL: {yandex_measures["wil"]:.2%}')
print(f'WIP: {yandex_measures["wip"]:.2f}')
print(f'BLEU: {bleu_score(reference_text, yandex_text):.2f}')
print(f'Cosine Similarity: {cosine_sim(reference_text, yandex_text):.2f}')
print(f'Levenshtein distance: {Levenshtein.distance(reference_text, yandex_text)}')

Yandex additional metrics:
MER: 25.23%
WIL: 33.33%
WIP: 0.67
BLEU: 0.67
Cosine Similarity: 0.84
Levenshtein distance: 537


*Общий вывод по проделанной работе: исходя из расчитанных метрик, можно утверждать, что с задачей транскрибации лучше справился Whisper*